In [2]:
import pandas as pd
import random
from IPython.display import clear_output


characters = {
    '5-star': ['Akuma Fuyu', 'Ajax', 'Arie', 'Benencino', 'Benivondo', 'Bromine', 'Cherri Bloom', 'Harumi', 'Lěng Dié', 'Liàng Huā', 'Rainbowrio', 'Unnamed Daylight', 'Unnamed Nightlight', 'Zayla'],
    '4-star': ['Adrinna', 'Caitylin', 'Hua Jingyi', 'Sire Bliss', 'Navia', 'Rosalyne', 'Scarlet Moon', 'Uviet'],
    '3-star': ['Akuma Akito', 'Amaris', 'Asiandra Dash', 'Blali Flammia', 'Blueberry Mane', 'Midnight Melody', 'Nightsky Artist', 'Sire Bliss'],
    '2-star': ['Torie', 'Aria', 'Asiandra Angel', 'Curlwhirl', 'Dagger', 'Dead Goth', 'Gunshot', 'Killerlane', 'Silkpetal', 'Toffee', 'Yokata Nao'],
    '1-star': ['Callie', 'Clova Pom', 'Misha', 'Moni', 'Morgana', 'Morgance', 'Osip', 'Yoshino Amaya'],
}

probabilities = {
    '5-star': 5,
    '4-star': 10,
    '3-star': 30,
    '2-star': 30,
    '1-star': 25
}


four_star_pity = 0
five_star_pity = 0

history = []

duplicates = {char: 0 for rarities in characters.values() for char in rarities}


def roll_pity(pity_type):
    """
    Simulates rolling for a character based on pity system.
    If pity counter hits its limit, it forces the respective rarity.
    """
    global four_star_pity, five_star_pity

    # 5-star pity check
    if pity_type == '5-star' and five_star_pity >= 90:
        five_star_pity = 0  # Reset 5-star pity after triggering a 5-star pull
        rarity = '5-star'
    elif pity_type == '4-star' and four_star_pity >= 45:
        four_star_pity = 0  # Reset 4-star pity after triggering a 4-star pull
        rarity = '4-star'
    else:
        roll = random.randint(1, 100)
        if roll <= probabilities['5-star']:
            rarity = '5-star'
        elif roll <= probabilities['5-star'] + probabilities['4-star']:
            rarity = '4-star'
        elif roll <= probabilities['5-star'] + probabilities['4-star'] + probabilities['3-star']:
            rarity = '3-star'
        elif roll <= probabilities['5-star'] + probabilities['4-star'] + probabilities['3-star'] + probabilities['2-star']:
            rarity = '2-star'
        else:
            rarity = '1-star'
    
    return rarity


def perform_pull(pulls=1):
    """
    Simulates pulling from the gacha system.
    Can do either one pull or ten pulls, accounting for pity system and duplicates.
    """
    global four_star_pity, five_star_pity, history
    
    for i in range(pulls):
        # Check if 5-star pity has reached 90 and force a 5-star if so
        if five_star_pity >= 90:
            rarity = '5-star'
            five_star_pity = 0  # Reset 5-star pity after pulling a 5-star
        else:
            # Regular roll, if no guaranteed 5-star
            roll = random.randint(1, 100)
            if roll <= probabilities['5-star']:
                rarity = '5-star'
                five_star_pity = 0  # Reset pity when a 5-star is pulled
            elif roll <= probabilities['5-star'] + probabilities['4-star']:
                rarity = '4-star'
                # 4-star doesn't reset 5-star pity counter
            elif roll <= probabilities['5-star'] + probabilities['4-star'] + probabilities['3-star']:
                rarity = '3-star'
            elif roll <= probabilities['5-star'] + probabilities['4-star'] + probabilities['3-star'] + probabilities['2-star']:
                rarity = '2-star'
            else:
                rarity = '1-star'

        # Increment pity counters after determining the rarity
        if rarity != '5-star':  # Only increment pity if it's not a forced 5-star
            four_star_pity += 1
            five_star_pity += 1
        
        # Choose character based on rarity
        if rarity == '5-star':
            character = random.choice(characters['5-star'])
        elif rarity == '4-star':
            character = random.choice(characters['4-star'])
        elif rarity == '3-star':
            character = random.choice(characters['3-star'])
        elif rarity == '2-star':
            character = random.choice(characters['2-star'])
        else:
            character = random.choice(characters['1-star'])

        # Track duplicates
        duplicates[character] += 1
        history.append({
            'Pull': len(history) + 1,
            'Rarity': rarity,
            'Character': character,
            'Duplicates': duplicates[character]
        })


def display_history():
    """
    Display the pull history as a dataframe without index, and center all text.
    """
    df = pd.DataFrame(history)
    display(df.style.set_properties(**{'text-align': 'center'}).hide(axis="index"))

def display_recent_pulls(pulls=1):
    """
    Display the most recent pulls (either just one or the last 10 pulls), 
    without index, and center all text.
    """
    # Display the most recent 'pulls' number of pulls (1 or 10)
    df = pd.DataFrame(history[-pulls:])  # Show the last 'pulls' pulls
    display(df.style.set_properties(**{'text-align': 'center'}).hide(axis="index"))

def display_characters():
    """
    Display all characters sorted by rarity and alphabetically, with the number of duplicates, in a dataframe with centered text.
    """
    # Create a list of tuples containing (rarity, character, count)
    character_counts = []
    for rarity, char_list in characters.items():
        for char in char_list:
            character_counts.append((rarity, char, duplicates[char]))

    # Sort by rarity first (5-star > 4-star > 3-star > 2-star > 1-star), then alphabetically
    rarity_order = {'5-star': 0, '4-star': 1, '3-star': 2, '2-star': 3, '1-star': 4}
    character_counts.sort(key=lambda x: (rarity_order[x[0]], x[1]))

    # Create the dataframe
    df = pd.DataFrame(character_counts, columns=['Rarity', 'Character', 'Duplicates'])

    # Display the dataframe with centered text
    display(df.style.set_properties(**{'text-align': 'center'}).hide(axis="index"))
        

def gacha_simulator():
    """
    The main loop for interacting with the Gacha simulator.
    """
    global four_star_pity, five_star_pity
    
    while True:
        user_input = input("What would you like to do? 1 for one pull, 10 for ten pulls, 'history' to view full history, 'characters' to view characters, 'quit' to exit.\n").lower()

        clear_output(wait=True)  # Clear previous output before displaying new output
        
        if user_input == 'quit':
            break
        elif user_input == 'history':
            display_history()  # Shows full history
        elif user_input == '1':
            perform_pull(1)  # Perform one pull
            display_recent_pulls(1)  # Show only the most recent pull
        elif user_input == '10':
            perform_pull(10)  # Perform ten pulls
            display_recent_pulls(10)  # Show all 10 pulls
        elif user_input == 'characters':
            display_characters()
        else:
            print("Invalid input. Please type 1, 10, 'history', 'characters' or 'quit'.")
        
        print(f"Pity: {five_star_pity}")


gacha_simulator()

Pull,Rarity,Character,Duplicates
81,4-star,Scarlet Moon,2
82,2-star,Toffee,5
83,3-star,Nightsky Artist,2
84,2-star,Curlwhirl,3
85,5-star,Benencino,1
86,1-star,Morgance,2
87,1-star,Misha,7
88,3-star,Amaris,7
89,2-star,Torie,2
90,2-star,Toffee,6


Pity: 5
What would you like to do? 1 for one pull, 10 for ten pulls, 'history' to view full history, 'characters' to view characters, 'quit' to exit.
quit


# Pain

In [ ]:
import pandas as pd
import random
from IPython.display import clear_output


characters = {
    '5-star': ['Akuma Fuyu', 'Ajax', 'Arie', 'Benencino', 'Benivondo', 'Bromine', 'Cherri Bloom', 'Harumi', 'Lěng Dié', 'Liàng Huā', 'Rainbowrio', 'Unnamed Daylight', 'Unnamed Nightlight', 'Zayla'],
    '4-star': ['Adrinna', 'Caitylin', 'Hua Jingyi', 'Sire Bliss', 'Navia', 'Rosalyne', 'Scarlet Moon', 'Uviet'],
    '3-star': ['Akuma Akito', 'Amaris', 'Asiandra Dash', 'Blali Flammia', 'Blueberry Mane', 'Midnight Melody', 'Nightsky Artist', 'Sire Bliss'],
    '2-star': ['Torie', 'Aria', 'Asiandra Angel', 'Curlwhirl', 'Dagger', 'Dead Goth', 'Gunshot', 'Killerlane', 'Silkpetal', 'Toffee', 'Yokata Nao'],
    '1-star': ['Callie', 'Clova Pom', 'Misha', 'Moni', 'Morgana', 'Morgance', 'Osip', 'Yoshino Amaya'],
}

probabilities = {
    '5-star': 5,
    '4-star': 10,
    '3-star': 30,
    '2-star': 30,
    '1-star': 25
}


four_star_pity = 0
five_star_pity = 0

history = []

duplicates = {char: 0 for rarities in characters.values() for char in rarities}


def roll_pity(pity_type):
    """
    Simulates rolling for a character based on pity system.
    If pity counter hits its limit, it forces the respective rarity.
    """
    global four_star_pity, five_star_pity

    # 5-star pity check
    if pity_type == '5-star' and five_star_pity >= 90:
        five_star_pity = 0  # Reset 5-star pity after triggering a 5-star pull
        rarity = '5-star'
    elif pity_type == '4-star' and four_star_pity >= 45:
        four_star_pity = 0  # Reset 4-star pity after triggering a 4-star pull
        rarity = '4-star'
    else:
        roll = random.randint(1, 100)
        if roll <= probabilities['5-star']:
            rarity = '5-star'
        elif roll <= probabilities['5-star'] + probabilities['4-star']:
            rarity = '4-star'
        elif roll <= probabilities['5-star'] + probabilities['4-star'] + probabilities['3-star']:
            rarity = '3-star'
        elif roll <= probabilities['5-star'] + probabilities['4-star'] + probabilities['3-star'] + probabilities['2-star']:
            rarity = '2-star'
        else:
            rarity = '1-star'
    
    return rarity


def perform_pull(pulls=1):
    """
    Simulates pulling from the gacha system.
    Can do either one pull or ten pulls, accounting for pity system and duplicates.
    """
    global four_star_pity, five_star_pity, history
    
    for i in range(pulls):
        # 10-pull guaranteed 4-star handling
        if pulls == 10:
            # Guarantee a 4-star if the 4-star pity counter reaches 45
            if four_star_pity >= 45:
                rarity = '4-star'  # Forced 4-star pull
                four_star_pity = 0  # Reset 4-star pity after this pull
            else:
                # Random roll for the 10-pull sequence (including forced 4-star)
                roll = random.randint(1, 100)
                if roll <= probabilities['5-star']:
                    rarity = '5-star'
                    five_star_pity = 0  # Reset 5-star pity after pulling a 5-star
                elif roll <= probabilities['5-star'] + probabilities['4-star']:
                    rarity = '4-star'
                    four_star_pity = 0  # Reset 4-star pity after pulling a 4-star
                else:
                    rarity = '3-star'
        else:
            # For 1-pulls, standard rolling and pity management
            if five_star_pity >= 90:
                rarity = '5-star'
                five_star_pity = 0  # Reset 5-star pity after pulling a 5-star
            elif four_star_pity >= 45:
                rarity = '4-star'
                four_star_pity = 0  # Reset 4-star pity after pulling a 4-star
            else:
                # Regular random roll for rarity
                roll = random.randint(1, 100)
                if roll <= probabilities['5-star']:
                    rarity = '5-star'
                    five_star_pity = 0  # Reset 5-star pity after pulling a 5-star
                elif roll <= probabilities['5-star'] + probabilities['4-star']:
                    rarity = '4-star'
                    four_star_pity = 0  # Reset 4-star pity after pulling a 4-star
                elif roll <= probabilities['5-star'] + probabilities['4-star'] + probabilities['3-star']:
                    rarity = '3-star'
                elif roll <= probabilities['5-star'] + probabilities['4-star'] + probabilities['3-star'] + probabilities['2-star']:
                    rarity = '2-star'
                else:
                    rarity = '1-star'

        # Increment pity counters after determining the rarity (skip for forced 5-star or 4-star)
        if rarity != '5-star' and rarity != '4-star':  # Only increment pity if it's not a forced 5-star or 4-star
            four_star_pity += 1
            five_star_pity += 1
        
        # Choose character based on rarity
        if rarity == '5-star':
            character = random.choice(characters['5-star'])
        elif rarity == '4-star':
            character = random.choice(characters['4-star'])
        elif rarity == '3-star':
            character = random.choice(characters['3-star'])
        elif rarity == '2-star':
            character = random.choice(characters['2-star'])
        else:
            character = random.choice(characters['1-star'])

        # Track duplicates
        duplicates[character] += 1
        history.append({
            'Pull': len(history) + 1,
            'Rarity': rarity,
            'Character': character,
            'Duplicates': duplicates[character]
        })


def display_history():
    """
    Display the pull history as a dataframe without index, and center all text.
    """
    df = pd.DataFrame(history)
    display(df.style.set_properties(**{'text-align': 'center'}).hide(axis="index"))

def display_recent_pulls(pulls=1):
    """
    Display the most recent pulls (either just one or the last 10 pulls), 
    without index, and center all text.
    """
    # Display the most recent 'pulls' number of pulls (1 or 10)
    df = pd.DataFrame(history[-pulls:])  # Show the last 'pulls' pulls
    display(df.style.set_properties(**{'text-align': 'center'}).hide(axis="index"))

def display_characters():
    """
    Display all characters sorted by rarity and alphabetically, with the number of duplicates, in a dataframe with centered text.
    """
    # Create a list of tuples containing (rarity, character, count)
    character_counts = []
    for rarity, char_list in characters.items():
        for char in char_list:
            character_counts.append((rarity, char, duplicates[char]))

    # Sort by rarity first (5-star > 4-star > 3-star > 2-star > 1-star), then alphabetically
    rarity_order = {'5-star': 0, '4-star': 1, '3-star': 2, '2-star': 3, '1-star': 4}
    character_counts.sort(key=lambda x: (rarity_order[x[0]], x[1]))

    # Create the dataframe
    df = pd.DataFrame(character_counts, columns=['Rarity', 'Character', 'Duplicates'])

    # Display the dataframe with centered text
    display(df.style.set_properties(**{'text-align': 'center'}).hide(axis="index"))
        

def gacha_simulator():
    """
    The main loop for interacting with the Gacha simulator.
    """
    global four_star_pity, five_star_pity
    
    while True:
        user_input = input("What would you like to do? 1 for one pull, 10 for ten pulls, 'history' to view full history, 'characters' to view characters, 'quit' to exit.\n").lower()

        clear_output(wait=True)  # Clear previous output before displaying new output
        
        if user_input == 'quit':
            break
        elif user_input == 'history':
            display_history()  # Shows full history
        elif user_input == '1':
            perform_pull(1)  # Perform one pull
            display_recent_pulls(1)  # Show only the most recent pull
        elif user_input == '10':
            perform_pull(10)  # Perform ten pulls
            display_recent_pulls(10)  # Show all 10 pulls
        elif user_input == 'characters':
            display_characters()
        else:
            print("Invalid input. Please type 1, 10, 'history', 'characters' or 'quit'.")
        
        print(f"Pity: {five_star_pity}")
        print(f"4 Pity: {four_star_pity}")


gacha_simulator()

Pull,Rarity,Character,Duplicates
11,3-star,Sire Bliss,4
12,3-star,Nightsky Artist,3
13,3-star,Amaris,3
14,4-star,Sire Bliss,5
15,3-star,Blueberry Mane,2
16,3-star,Blueberry Mane,3
17,5-star,Ajax,1
18,4-star,Adrinna,1
19,3-star,Blueberry Mane,4
20,3-star,Blali Flammia,3


Pity: 2
4 Pity: 2
